In [49]:
import requests
import gzip
import xml.etree.ElementTree as ET
import os
import zipfile
import re
import random
import string
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [4]:
##create_ted_xml_files!
# Function to download and extract ZIP files
def download_and_extract(url, output_dir):
    response = requests.get(url)
    zip_file = os.path.join(output_dir, 'temp.zip')
    
    with open(zip_file, 'wb') as f:
        f.write(response.content)
    
    # Extract ZIP file
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(output_dir)

    # Delete the temporary ZIP file
    os.remove(zip_file)




In [5]:
# Example URLs for downloading
heb_url = 'https://object.pouta.csc.fi/OPUS-TED2020/v1/xml/he.zip'
uk_url = 'https://object.pouta.csc.fi/OPUS-TED2020/v1/xml/uk.zip'
fi_url = 'https://object.pouta.csc.fi/OPUS-TED2020/v1/xml/fi.zip'

# Directories to store the extracted files
output_dir = 'ted_xml_files'

# Create the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Download and extract the files
download_and_extract(heb_url, output_dir)
download_and_extract(uk_url, output_dir)
download_and_extract(fi_url, output_dir)


In [6]:
def decompress_alignments(input_file, output_file):
    with gzip.open(input_file, 'rb') as f_in:
        with open(output_file, 'wb') as f_out:
            f_out.write(f_in.read())

In [7]:
he_uk_pairs = 'he-uk.xml'
he_uk_alignments = 'he_uk_alignments.xml'
decompress_alignments(he_uk_pairs, he_uk_alignments)

In [8]:
he_fi_pairs = 'he-fi.xml'
he_fi_alignments = 'he_fi_alignments.xml'
decompress_alignments(he_fi_pairs, he_fi_alignments)

In [9]:
def extract_sentences_from_xml(folder_path):
    all_sentences = {}
    
    # Iterate through each XML file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".xml"):
            file_path = os.path.join(folder_path, filename)
            
            # Parse the XML file
            tree = ET.parse(file_path)
            root = tree.getroot()
            
            # Dictionary to temporarily store sentences
            temp_sentences = {}
            
            # Extract sentences from the XML file
            for sentence in root.findall(".//s"):
                sentence_id = sentence.get('id')
                words = [word.text for word in sentence.findall('.//w')]
                full_sentence = ' '.join(words)
                
                # Temporarily store sentences by their IDs (even sub-IDs)
                temp_sentences[sentence_id] = full_sentence
            
            # Now merge sub-sentences into the main sentence
            merged_sentences = {}
            for sentence_id, sentence in temp_sentences.items():
                main_id = sentence_id.split('.')[0]  # Get the main sentence ID (e.g., '1')
                if main_id not in merged_sentences:
                    merged_sentences[main_id] = sentence
                else:
                    # Append the sub-sentence to the main sentence
                    merged_sentences[main_id] += ' ' + sentence
            
            # Store the final merged sentences in the all_sentences dictionary
            for sentence_id, sentence in merged_sentences.items():
                all_sentences[(filename, sentence_id)] = sentence
    
    return all_sentences

In [10]:
# Folder path where your XML files are stored
fi_folder_path = '/home/shirmash/research_methods/ted_xml_files/TED2020/xml/fi'
he_folder_path = '/home/shirmash/research_methods/ted_xml_files/TED2020/xml/he'
uk_folder_path = '/home/shirmash/research_methods/ted_xml_files/TED2020/xml/uk'

In [11]:
fi_all_sentences = extract_sentences_from_xml(fi_folder_path)
he_all_sentences = extract_sentences_from_xml(he_folder_path)
uk_all_sentences = extract_sentences_from_xml(uk_folder_path)

In [12]:
for idx, (key, value) in enumerate(he_all_sentences.items()):
    if idx == 5:
        break
    print(key, value)

('ted2020-553.xml', '1') אני כאן כי יש לי הודעה חשובה מאוד . אני חושב שמצאנו את הגורם החשוב ביותר להצלחה .
('ted2020-553.xml', '2') מצאו אותו לא רחוק מכאן , בסטנפורד .
('ted2020-553.xml', '3') פרופסור לפסיכולוגיה לקח ילדים בני ארבע והשאיר אותם לגמרי לבדם בחדר .
('ted2020-553.xml', '4') והוא אמר לילד , לילד בן הארבע , " ג ' וני , אני הולך להשאיר אותך כאן עם מרשמלו ל-15 דקות .
('ted2020-553.xml', '5') אם כשאני אחזור המרשמלו יהיה כאן , תקבל עוד אחד . ואז יהיו לך שניים . "


In [13]:
for idx, (key, value) in enumerate(fi_all_sentences.items()):
    if idx == 5:
        break
    print(key, value)

('ted2020-553.xml', '1') Olen täällä , koska minulla on tärkeä viesti . Olemme luullakseni löytäneet tärkeimmän tekijän menestykseen .
('ted2020-553.xml', '2') Ja se löydettiin tässä lähellä , Stanfordissa .
('ted2020-553.xml', '3') Psykologian professori laittoi neljävuotiaita lapsia huoneeseen ja jätti heidät itsekseen .
('ted2020-553.xml', '4') Hän kertoi lapsille : " Johnny , aion nyt lähteä ja jättää sinut yksin vaahtokarkin kanssa 15 minuutiksi . "
('ted2020-553.xml', '5') " Jos karkki on tässä , kun tulen takaisin , saat toisenkin . Eli saat kaksi . "


In [14]:
def combine_dicts(dict1, dict2):
    # Create a new dictionary where the key is present in both dict1 and dict2
    combined_dict = {}

    for key in dict1:
        if key in dict2:
            combined_dict[key] = [dict1[key], dict2[key]]

    return combined_dict

In [15]:
# Create the mappings
he_uk_dict= combine_dicts(he_all_sentences, uk_all_sentences)
he_fi_dict= combine_dicts(he_all_sentences, fi_all_sentences)
# Output the results
print("Hebrew - Ukrainian Dictionary:")
print(len(he_uk_dict))

print("\nHebrew - Finnish Dictionary:")
print(len(he_fi_dict))

Hebrew - Ukrainian Dictionary:
188241

Hebrew - Finnish Dictionary:
43751


download verion after filtering by embedding similarity

In [50]:
with open("filtered_he_uk_dict_by_sim.pkl", "rb") as f:
    loaded_he_uk_dict = pickle.load(f)

with open("filtered_he_fi_dict_by_sim.pkl", "rb") as f:
    loaded_he_fi_dict = pickle.load(f)

In [16]:

def clean_sentence(sentence):
    # Remove punctuation for language checking but leave spaces
    return ''.join(ch for ch in sentence if ch not in string.punctuation)


def contains_english(sentence):
    cleaned_sentence = clean_sentence(sentence)
    return bool(re.search(r'[A-Za-z]', cleaned_sentence))  # Contains any English letters




In [17]:

# Function to filter sentences in the given dictionaries
def filter_valid_sentences_by_english(data_dict):
    valid_sentences = {}
    for identifier, sentences in data_dict.items():
        hebrew_sentence, second_sentence = sentences
        # Check that neither sentence contains English characters
        if not contains_english(hebrew_sentence):
                valid_sentences[identifier] = sentences
        
    
    return valid_sentences

In [46]:
def filter_dict(input_dict, thres=0.75):
    # Load the Sentence Transformer model
    model = SentenceTransformer('intfloat/multilingual-e5-large').to('cuda')
    
    # Dictionary to hold valid sentences
    valid_dict = {}
    
    for key, sentences in input_dict.items():
        # Encode the pair of sentences
        embeddings = model.encode(sentences, normalize_embeddings=True)
        
        # Compute cosine similarity
        dist = cosine_similarity(embeddings[0].reshape(1, -1), embeddings[1].reshape(1, -1))[0][0]
        
        # Retain only the sentences that meet the threshold
        if dist >= thres:
            valid_dict[key] = sentences
    
    return valid_dict

In [47]:
def select_random_sample_with_matching_keys(dict1, dict2, sample_size=50000):
    # Find common keys between both dictionaries
    common_keys = set(dict1.keys()).intersection(set(dict2.keys()))
    
    # If there are fewer common keys than the sample size, return all
    if len(common_keys) < sample_size:
        print(f"Only {len(common_keys)} common keys found, returning all.")
        sampled_keys = list(common_keys)
    else:
        # Randomly sample the required number of keys from the common keys
        sampled_keys = random.sample(common_keys, sample_size)
    
    # Create new dictionaries with sampled keys, ensuring keys are ordered
    sampled_dict1 = {key: dict1[key] for key in sorted(sampled_keys)}
    sampled_dict2 = {key: dict2[key] for key in sorted(sampled_keys)}
    
    return sampled_dict1, sampled_dict2

In [55]:
filtered_he_uk_dict = filter_valid_sentences_by_english(loaded_he_uk_dict)
filtered_he_fi_dict = filter_valid_sentences_by_english(loaded_he_fi_dict)

#filtered_he_uk_dict = filter_dict(filtered_he_uk_dict, thres=0.75)
# filtered_he_fi_dict = filter_dict(filtered_he_fi_dict, thres=0.75)


sampled_he_uk_dict,sampled_he_fi_dict = select_random_sample_with_matching_keys(filtered_he_uk_dict,filtered_he_fi_dict, 20000)

# The dictionaries now contain 50,000 randomly selected valid sentences or fewer
print("Sampled Hebrew-Ukraine Dictionary:", len(sampled_he_uk_dict))
print("Sampled Hebrew-Finnish Dictionary:", len(sampled_he_fi_dict))


Sampled Hebrew-Ukraine Dictionary: 20000
Sampled Hebrew-Finnish Dictionary: 20000


/tmp/ipykernel_1460133/2559842779.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  sampled_keys = random.sample(common_keys, sample_size)


In [57]:
for idx, (k, value) in enumerate(sampled_he_uk_dict.items()):
    if idx == 5:
        break
    print(k, value)

('ted2020-10.xml', '1') ['עם כל החששות הלגיטימיים על איידס , שפעת העופות -- ואנו נשמע על זה מד " ר בריליאנט המבריק מאוחר יותר -- אני אדבר על מגיפות אחרות , מחלות לב , כלי דם , סוכרת ויתר לחץ דם שהן ניתנות למניעה לחלוטין עבור לפחות 95 אחוז של אנשים , על ידי שינוי תזונה ואורח חיים .', 'Враховуючи всю резонність стурбованості про СНІД і пташиний грип — ми ще почуємо про це від геніального Доктора Бриллиант сьогодні — я хотів би поговорити про іншу пандемію , а саме , про серцево-судинні захворювання , діабет , гіпертонію — захворювання , що можна повністю уникнути принаймні в 95 % людей , змінивши дієту і спосіб життя .']
('ted2020-10.xml', '10') ['אבל באסיה מתחילים לאכול כמונו וזו הסיבה שהם מתחילים לחלות כמונו .', 'Але люди в Азії починають харчуватися як ми , і тому вони починають хворіти як ми .']
('ted2020-10.xml', '12') ["הסלטים במקדונלד ' ס מקורם בעבודה זו . בפפסי , שני שליש מהצמיחה מגיע מהאוכל הבריא .", 'Салати , які ви бачите в МакДональдсах , — це результат тієї роботи — у них не

In [59]:
for idx, (k, value) in enumerate(sampled_he_fi_dict.items()):
    if idx == 5:
        break
    print(k, value)

('ted2020-10.xml', '1') ['עם כל החששות הלגיטימיים על איידס , שפעת העופות -- ואנו נשמע על זה מד " ר בריליאנט המבריק מאוחר יותר -- אני אדבר על מגיפות אחרות , מחלות לב , כלי דם , סוכרת ויתר לחץ דם שהן ניתנות למניעה לחלוטין עבור לפחות 95 אחוז של אנשים , על ידי שינוי תזונה ואורח חיים .', 'Näiden oikeutettujen AIDS- ja lintuinfluenssahuolien -- ja kuulemme näistä lisää myöhemmin loistavalta tohtori Brilliantilta -- tahdon puhua teille muista pandemioista , joita ovat sydän- ja verisuonitaudit , diabetes , verenpainetauti -- kaikki täysin ehkäistävissä ainakin 95 prosentissa ihmisistä vain muuttamalla ruokailutottumuksia ja elämäntapaa .']
('ted2020-10.xml', '10') ['אבל באסיה מתחילים לאכול כמונו וזו הסיבה שהם מתחילים לחלות כמונו .', 'Mutta ihmiset Aasiassa ovat alkaneet syödä kuin me , minkä takia he ovat alkaneet sairastua kuten me .']
('ted2020-10.xml', '12') ["הסלטים במקדונלד ' ס מקורם בעבודה זו . בפפסי , שני שליש מהצמיחה מגיע מהאוכל הבריא .", "Salaatit , jotka näette McDonald ' sissa , ov

In [62]:
import pickle
with open("final_he_fi_dict.pkl", "wb") as f:
    pickle.dump(sampled_he_fi_dict, f)

with open("final_he_uk_dict.pkl", "wb") as f:
    pickle.dump(sampled_he_uk_dict, f)